In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load nutrition data
nutritions <- read.csv("../input/nutrition-clean/nutrition_cleaned.csv")

Data Preparation
Prior to clustering data, you may want to remove or estimate missing data and rescale variables for comparability.

In [ ]:
# Prepare Data
mydata <- nutritions[,4:17]
mydata <- na.omit(mydata) # listwise deletion of missing
mydata <- scale(mydata) # standardize variables

In [ ]:
mydata

Partitioning
K-means clustering is the most popular partitioning method. It requires the analyst to specify the number of clusters to extract. A plot of the within groups sum of squares by number of clusters extracted can help determine the appropriate number of clusters. The analyst looks for a bend in the plot similar to a scree test in factor analysis. See Everitt & Hothorn (pg. 251).

In [ ]:
# Determine number of clusters
wss <- (nrow(mydata)-1)*sum(apply(mydata,2,var))
for (i in 2:15) wss[i] <- sum(kmeans(mydata,
   centers=i)$withinss)
plot(1:15, wss, type="b", xlab="Number of Clusters",
  ylab="Within groups sum of squares")

In [ ]:
# K-Means Cluster Analysis
fit <- kmeans(mydata, 5) # 5 cluster solution
# get cluster means
aggregate(mydata,by=list(fit$cluster),FUN=mean)
# append cluster assignment
mydata <- data.frame(mydata, fit$cluster)



A robust version of K-means based on mediods can be invoked by using pam( ) instead of kmeans( ). The function pamk( ) in the fpc package is a wrapper for pam that also prints the suggested number of clusters based on optimum average silhouette width.

Hierarchical Agglomerative
There are a wide range of hierarchical clustering approaches. I have had good luck with Ward's method described below.

In [ ]:
# Ward Hierarchical Clustering
d <- dist(mydata, method = "euclidean") # distance matrix
fit <- hclust(d, method="ward")
plot(fit) # display dendogram
groups <- cutree(fit, k=5) # cut tree into 5 clusters
# draw dendogram with red borders around the 5 clusters
rect.hclust(fit, k=5, border="red")

The pvclust( ) function in the pvclust package provides p-values for hierarchical clustering based on multiscale bootstrap resampling. Clusters that are highly supported by the data will have large p values. Interpretation details are provided Suzuki. Be aware that pvclust clusters columns, not rows. Transpose your data before using.

In [ ]:
# Ward Hierarchical Clustering with Bootstrapped p values
library(pvclust)
fit <- pvclust(mydata, method.hclust="ward",
   method.dist="euclidean")
plot(fit) # dendogram with p values
# add rectangles around groups highly supported by the data
pvrect(fit, alpha=.95)

Model Based
Model based approaches assume a variety of data models and apply maximum likelihood estimation and Bayes criteria to identify the most likely model and number of clusters. Specifically, the Mclust( ) function in the mclust package selects the optimal model according to BIC for EM initialized by hierarchical clustering for parameterized Gaussian mixture models. (phew!). One chooses the model and number of clusters with the largest BIC. See help(mclustModelNames) to details on the model chosen as best.

In [ ]:
# Model Based Clustering
library(mclust)
fit <- Mclust(mydata)
plot(fit) # plot results
summary(fit) # display the best model

Plotting Cluster Solutions
It is always a good idea to look at the cluster results.

In [ ]:
# K-Means Clustering with 5 clusters
fit <- kmeans(mydata, 5)

# Cluster Plot against 1st 2 principal components

# vary parameters for most readable graph
library(cluster)
clusplot(mydata, fit$cluster, color=TRUE, shade=TRUE,
   labels=2, lines=0)

# Centroid Plot against 1st 2 discriminant functions
library(fpc)
plotcluster(mydata, fit$cluster)

Validating cluster solutions
The function cluster.stats() in the fpc package provides a mechanism for comparing the similarity of two cluster solutions using a variety of validation criteria (Hubert's gamma coefficient, the Dunn index and the corrected rand index)

In [ ]:
# comparing 2 cluster solutions
library(fpc)
cluster.stats(d, fit1$cluster, fit2$cluster)

In [ ]:
where d is a distance matrix among objects, and fit1$cluster and fit$cluster are integer vectors containing classification results from two different clusterings of the same data.

In [ ]:
# set plot layout 
par(mfrow=c(3,2),
     mar=c(2,2,2,1), oma=c(0,0,0,0) # margin: buttom left, top, right
     )

# training data with first four attributes
trainingData <-  mydata
# 2D plot with two attributes: Sepal.Length Sepal.Width 
plotData <- mydata[,c(1,2)]

# set number of clusters which is a required argument for some clustering algorithms 
numberOfClusters <- 5

#  -------- K-Means ----------

model <- kmeans(trainingData, numberOfClusters)
plot(plotData, col=model$cluster, main="K-Means")
# point center of two attributes of plotData
points(model$centers[, c(1,3)], col=1:3, pch=8, cex=2)


#  -------- Fuzzy C-Means ----------

library(e1071)
# m - A number greater than 1 giving the degree of fuzzification.
model <- cmeans(trainingData, numberOfClusters, m=2, method="cmeans")
plot(plotData, col=model$cluster, main="Fuzzy C-Means")
points(model$centers[, c(1,3)], col=1:3, pch=8, cex=2)


# ------------ Mean Shift  -----------------

library(MeanShift)
model <- msClustering( t(as.matrix(trainingData)), h=0.91 )
plot(plotData, col=model$labels, main="MeanShift")


#  -------- Expectation-Maximization --------

library(mclust)
model <- Mclust(trainingData, numberOfClusters)
plot(plotData, col=model$classification, main="Expectation-Maximization")


#  -------- Density-based -------

library(fpc)
model <- dbscan(trainingData, eps=0.6, MinPts=4)
plot(plotData, col=4-model$cluster, main="Density-based")
mtext("Noise/outlier observations are coded as 0, and plotted in blue", cex=0.5)


# --------- Hierarchical ------

distance <- dist(trainingData, method="euclidean") 
hc <- hclust(distance, method="average")
model <- cutree(hc, numberOfClusters)
plot(plotData, col=model, main="Hierarchical")



# ---------- Self-Organising Maps -----

library(kohonen) 
som <- som(as.matrix(scale(trainingData)), somgrid(xdim=5, ydim=30, topo="hexagonal"))
## use hierarchical clustering to cluster the codebook vectors
model <- cutree(hclust(dist(som$codes[[1]])), numberOfClusters)
plot(plotData, col=model, main="Self-Organising Maps")


# ---------- Spectral ------- 

library(kernlab)
model <- specc(as.matrix(trainingData), centers=numberOfClusters)
plot(plotData, col=model, main="Spectral")